In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)

In [2]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST',
    train=True, 
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [3]:
train_set.shape

AttributeError: 'FashionMNIST' object has no attribute 'shape'

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    
    def forward(self, t):
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.fc1(t.reshape(-1, 12*4*4)))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        
        return t

In [ ]:
torch.set_grad_enabled(False)

In [ ]:
network = Network()

In [ ]:
sample = next(iter(train_set))

In [ ]:
image, label = sample

In [ ]:
batch = torch.unsqueeze(image, dim=0)
batch.shape

In [ ]:
pred = network(batch)

In [ ]:
# if we want the prediction to be probabilities (positive numbers adding up t0 1)
# use the softmax() in the output layer

pred

In [ ]:
label

In [ ]:
pred.argmax(dim=1)

In [ ]:
F.softmax(pred, dim=1)

In [ ]:
F.softmax(pred, dim=1).sum()

### Train the Model Using a Batch of Input Data

In [ ]:
data_loader = torch.utils.data.DataLoader(train_set, batch_size=10)

In [ ]:
batch = next(iter(data_loader))

images, labels = batch
images.shape, labels.shape

In [ ]:
preds = network(images)

In [ ]:
preds

In [ ]:
preds.argmax(dim=1)

In [ ]:
labels

In [ ]:
preds.argmax(dim=1).eq(labels).sum()

In [ ]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [ ]:
get_num_correct(preds, labels)

### The Training Process
1. Get batch to network
2. Pass batch to the network
3. Calculate the loss (difference between the prediction values and true values)
4. Calculate the gradient of the loss function with respect to the networks's weights
5. Update the weights using the gradients to reduce the loss
6. Repeat steps 1-5 until one epoch is completed (every batch has been passed through network)
7. Repeat steps 1-6 for as many epochs required to obtain the desired level of accuracy

In [ ]:
import torch.optim as optim
torch.set_grad_enabled(True)

### Calcualte the Loss

In [ ]:
loss = F.cross_entropy(preds, labels) # Calculate the loss
loss.item()

In [ ]:
print(network.conv1.weight.grad)

### Updating the Weights

In [ ]:
optimizer = optim.Adam(network.parameters(), lr=0.001)

In [ ]:
loss.item()

In [ ]:
get_num_correct(preds, labels)

In [ ]:
optimizer.step() # updating the weights

In [ ]:
preds = network(images)
loss = F.cross_entropy(preds, labels)

In [ ]:
loss.item()

In [ ]:
get_num_correct(preds, labels)

In [ ]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.001)

batch = next(iter(train_loader))
images, labels = batch

preds = network(images)
loss = F.cross_entropy(preds, labels)

loss.backward()
optimizer.step()

In [ ]:
print('loss1:', loss.item())
preds = network(images)
loss = F.cross_entropy(preds, labels)
print('loss2:', loss.item())